<a href="https://colab.research.google.com/github/maneeha/KGLLM/blob/main/Diabetes%20-%20Wikipedia%20to%20graph%20-%20RAG_With_Knowledge_graph(Neo4j).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-core

contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

# langchain-community
contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

# langchain
contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.#

In [5]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 24.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [8]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [9]:
from typing import Tuple, List, Optional

In [10]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [11]:
from langchain_core.runnables import ConfigurableField

In [12]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [13]:
import os

In [14]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [15]:
from langchain_community.vectorstores import Neo4jVector

In [16]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [17]:
NEO4J_URI="neo4j+s://e3a484ff.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=""

In [18]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [19]:
from langchain_community.graphs import Neo4jGraph

In [20]:
graph = Neo4jGraph()

In [21]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Diabetes_management").load()

In [22]:
len(raw_documents)

25

In [23]:
raw_documents[:3]

[Document(page_content='The term diabetes includes several different metabolic disorders that all, if left untreated, result in abnormally high concentrations of a sugar called glucose in the blood. Diabetes mellitus type 1 results when the pancreas no longer produces significant amounts of the hormone insulin, usually owing to the autoimmune destruction of the insulin-producing beta cells of the pancreas. Diabetes mellitus type 2, in contrast, is now thought to result from autoimmune attacks on the pancreas and/or insulin resistance. The pancreas of a person with type 2 diabetes may be producing normal or even abnormally large amounts of insulin. Other forms of diabetes mellitus, such as the various forms of maturity-onset diabetes of the young, may represent some combination of insufficient insulin production and insulin resistance. Some degree of insulin resistance may also be present in a person with type 1 diabetes.\nThe main goal of diabetes management and control is, as far as p

In [24]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [25]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [26]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [27]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [28]:
graph_documents

[GraphDocument(nodes=[Node(id='Diabetes', type='Metabolic disorder'), Node(id='Glucose', type='Sugar'), Node(id='Insulin', type='Hormone'), Node(id='Pancreas', type='Organ'), Node(id='Type 1 Diabetes', type='Diabetes mellitus'), Node(id='Type 2 Diabetes', type='Diabetes mellitus'), Node(id='Insulin Resistance', type='Condition'), Node(id='Maturity-Onset Diabetes Of The Young', type='Diabetes mellitus'), Node(id='Carbohydrate Metabolism', type='Metabolism'), Node(id='Insulin Replacement Therapy', type='Therapy'), Node(id='Dietary Modifications', type='Modification'), Node(id='Exercise', type='Activity'), Node(id='Blood Glucose', type='Blood measurement'), Node(id='Blood Pressure', type='Blood measurement'), Node(id='Lipids', type='Substance'), Node(id='Hba1C', type='Blood measurement'), Node(id='Hypoglycemia', type='Condition'), Node(id='Cardiovascular Disease', type='Disease'), Node(id='Microvascular Disease', type='Disease'), Node(id='Clinical Practice Guidelines', type='Guidelines'),

In [29]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [30]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"
default_cypher

'MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50'

In [31]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [32]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [33]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [34]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [35]:
from typing import Tuple, List, Optional

In [36]:
from langchain_community.vectorstores import Neo4jVector

In [37]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [38]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [39]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the diabetes care related entities that "
        "appear in the text",
    )


In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [41]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting symptoms and care related entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [42]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [43]:
entity_chain.invoke({"question": "At what age diabetes risk gets increased?"}).names

['diabetes risk', 'age']

In [44]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [45]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [46]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [47]:
print(structured_retriever("What Diabetes is?"))

Diabetes - RESULTS_IN -> Glucose
Diabetes - RELATED_TO -> Insulin
Diabetes - AFFECTS -> Carbohydrate Metabolism
Diabetes - AFFECTS -> Microvascular Disease
Diabetes - AFFECTS -> Eyes
Diabetes - AFFECTS -> Kidneys
Diabetes - AFFECTS -> Nerves
Diabetes - CAN_LEAD_TO -> Complications
Diabetes - CONTRIBUTES_TO -> Risk Factors
Diabetes - CAUSE -> Insulin Resistance
Diabetes - INCLUDE -> Stroke
Diabetes - INCLUDE -> Peripheral Artery Disease
Diabetes - DOUBLES_THE_RISK_OF -> Cardiovascular Disease
Diabetes - DUE_TO -> Coronary Artery Disease
Diabetes - RECOMMENDED_THAT_VISIT -> Optometrist
Diabetes - RECOMMENDED_THAT_VISIT -> Ophthalmologist
Diabetes - HIGH_CONCENTRATIONS -> Glucose
Diabetes - GOAL_OF_MANAGEMENT -> Carbohydrate Metabolism
Diabetes - AFFECTED_ORGAN -> Pancreas
Diabetes - CONTROL_GOAL -> Blood Glucose
Diabetes - CONTROL_GOAL -> Blood Pressure
Diabetes - CONTROL_GOAL -> Lipids
Diabetes - CONTROL_GOAL -> Hba1C
Diabetes - RISK_FACTOR -> Hypoglycemia
Diabetes - COMPLICATION -> Car

In [48]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [49]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [50]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [51]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [52]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [53]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [54]:
prompt = ChatPromptTemplate.from_template(template)

In [55]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [56]:
chain.invoke({"question": "how stress-related hormones affects diabetes and sugar level?"})

Search query: how stress-related hormones affects diabetes and sugar level?


'Stress-related hormones, such as adrenaline, can impact diabetes by causing an increase in blood sugar levels. This is because stress hormones can trigger the release of glucose into the bloodstream, leading to elevated sugar levels in individuals with diabetes.'

In [57]:
chain.invoke(
    {
        "question": "When diabetes gets worst",
        "chat_history": [("how stress-related hormones affects diabetes and sugar level?", "increases diabetes")],
    }
)

Search query: When does diabetes get worse?


'When diabetes gets worse, it can lead to various health complications affecting the cardiovascular system, eyes, kidneys, and nerves. It can also result in disorders such as diabetic retinopathy, diabetic nephropathy, and diabetic neuropathy.'